# Getting started with Azure AI Agent Service

## Running an existing agent
If you've already created an agent, you can run it with the following code. Simply provide your agent ID.

In [1]:
# Ask user for agent_id from the python notebook
agent_id = input("Enter agent_id: ")

In [2]:
# Ask user for their connection ID to their Foundry project
project_connection_string = input("Enter project connection string: ")

In [3]:
import os
import json
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery"
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

In [ ]:
# Verify that you can get the agent you already created
agent = project_client.agents.get_agent(agent_id)
print(agent)

In [ ]:
# Let's talk to it!

# Create a new conversation
thread = project_client.agents.create_thread()

while True:
    user_input = input("User >")
    print("User >", user_input)

    if user_input == "":
        break

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )

    print("Agent > ", end="")
    # Have agent process the conversation
    with project_client.agents.create_stream(
            thread_id=thread.id, assistant_id=agent.id
        ) as stream:
            for event_type, event_data, func_return in stream:
                # print(event_data)
                if event_type == "thread.run.step.delta":
                    for tool_call in event_data.delta.step_details.tool_calls:
                        if tool_call.type == "bing_grounding":
                            print(tool_call['bing_grounding']['requesturl'])
                if event_type == "thread.message.delta":
                    print(event_data.delta.content[0].text.value, end="")
    print()

